In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import datetime

#### 네이버 환율 일별시세

- 미국(USD), 유럽연합(EUR), 일본(JPY), 중국(CNY) 통화의 매매기준율 추출

In [2]:
def exchange_day_sise_crawler(country, base_url):

    temp_dict = {}

    usaList = []
    eurList = []
    jpyList = []
    cnyList = []

    for country_name, country_code in country.items():
        dateList = []
        for i in range(1,10):
            params = {
                'marketindexCd': country_code,
            }

            resp = requests.get(base_url, params=params)
            soup = BeautifulSoup(resp.text)

            _iframe = soup.find_all('iframe')
            day_sise = urljoin(base_url,_iframe[1].get('src'))
            day_sise = day_sise+'&page='+str(i)

            resp2 = requests.get(day_sise)
            soup2 = BeautifulSoup(resp2.text,'lxml')

            day_body = soup2.select('tbody tr')
            
            # 각 나라별 환율정보 저장
            for body in day_body:
                dateList.append(body.find('td', class_='date').text.replace('.','-'))

                if country_name == 'USA':
                    usaList.append(body.find('td', class_='num').text)

                elif country_name == 'EUR':
                    eurList.append(body.find('td', class_='num').text)

                elif country_name == 'JPY':
                    jpyList.append(body.find('td', class_='num').text)

                elif country_name == 'CNY':
                    cnyList.append(body.find('td', class_='num').text)
    
    df1 = pd.DataFrame({'날짜' : dateList,
                        'USA' : usaList,
                        'EUR' : eurList,
                        'JPY' : jpyList,
                        'CNY' : cnyList})
    
    return df1

#### 수집해야할 모든날짜 DataFrame 만들기 (공휴일 포함)

In [3]:
def all_date_range_df(date_range):
    
    df2 = pd.DataFrame({'날짜':date_range})
    
    return df2

#### 두개의 DataFrame 병합

In [4]:
def df_outer_join_merge(df1, df2):
    
    # 두 df을 '날짜'키 값을 기준으로 outer_join 병합
    df_OUTER_JOIN = pd.merge(df1, df2, left_on='날짜', right_on='날짜', how='outer', sort=True)
    
    # 두 값중 결측값을 앞 방향으로 채우기 (공휴일에 없는 환율정보는 전날을 기준으로 결측값 채우기)
    # https://rfriend.tistory.com/262
    df_finally = df_OUTER_JOIN.fillna(method='ffill')
    
    return df_finally

In [5]:
def save_df_to_csv(df_finally):
    df_finally.to_csv('naver_exchange_crawling.csv', encoding='utf-8', index=False)

In [6]:
country = {'USA':'FX_USDKRW', 'EUR':'FX_EURKRW', 'JPY':'FX_JPYKRW', 'CNY':'FX_CNYKRW'}
base_url = 'https://finance.naver.com/marketindex/exchangeDetail.nhn'

date_range = (pd.date_range(start='20200501', end=datetime.date.today())).strftime("%Y-%m-%d").tolist()

ExchangeDf = exchange_day_sise_crawler(country, base_url)
DateDf = all_date_range_df(date_range)
FinallyDf = df_outer_join_merge(ExchangeDf, DateDf)
#     save_df_to_csv(FinallyDf)

print('=====END Exchange Crawling=====')

=====END Exchange Crawling=====
